In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functorch import combine_state_for_ensemble

In [2]:
class SimNorm(nn.Module):
	"""
	Simplicial normalization.
	Adapted from https://arxiv.org/abs/2204.00616.
	"""
	
	def __init__(self, cfg):
		super().__init__()
		self.dim = cfg.simnorm_dim
	
	def forward(self, x):
		shp = x.shape
		x = x.view(*shp[:-1], -1, self.dim)
		x = F.softmax(x, dim=-1)
		return x.view(*shp)
		
	def __repr__(self):
		return f"SimNorm(dim={self.dim})"

In [3]:
import collections

from dm_control import mujoco
from dm_control.rl import control
from dm_control.suite import base
from dm_control.suite import common
from dm_control.suite.utils import randomizers
from dm_control.utils import containers
from dm_control.utils import rewards


_DEFAULT_TIME_LIMIT = 25
_CONTROL_TIMESTEP = .025

# Minimal height of torso over foot above which stand reward is 1.
_STAND_HEIGHT = 1.2

# Horizontal speeds (meters/second) above which move reward is 1.
_WALK_SPEED = 1
_RUN_SPEED = 8


SUITE = containers.TaggedTasks()

/home/pipaek/miniconda3/envs/tdmpc2/lib/python3.9/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


In [4]:
def get_model_and_assets():
  """Returns a tuple containing the model XML string and a dict of assets."""
  return common.read_model('walker.xml'), common.ASSETS

In [5]:
class Physics(mujoco.Physics):
  """Physics simulation with additional features for the Walker domain."""

  def torso_upright(self):
    """Returns projection from z-axes of torso to the z-axes of world."""
    return self.named.data.xmat['torso', 'zz']

  def torso_height(self):
    """Returns the height of the torso."""
    return self.named.data.xpos['torso', 'z']

  def horizontal_velocity(self):
    """Returns the horizontal velocity of the center-of-mass."""
    return self.named.data.sensordata['torso_subtreelinvel'][0]

  def orientations(self):
    """Returns planar orientations of all bodies."""
    return self.named.data.xmat[1:, ['xx', 'xz']].ravel()


In [6]:
physics = Physics.from_xml_string(*get_model_and_assets())

In [7]:
physics.orientations()

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [8]:
physics.torso_height()

1.3

In [9]:
physics.velocity()

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
physics.named.data

<dm_control.mujoco.index.make_struct_indexer.<locals>.StructIndexer at 0x7fef0dc12370>

In [12]:
physics.named.data.xpos

FieldIndexer(xpos):
                x         y         z         
0       world [ 0         0         0       ]
1       torso [ 0         0         1.3     ]
2 right_thigh [ 0        -0.05      1       ]
3   right_leg [ 0        -0.05      0.3     ]
4  right_foot [ 0.06     -0.05      0.05    ]
5  left_thigh [ 0         0.05      1       ]
6    left_leg [ 0         0.05      0.3     ]
7   left_foot [ 0.06      0.05      0.05    ]

In [13]:
physics.named.data.xmat

FieldIndexer(xmat):
                xx        xy        xz        yx        yy        yz        zx        zy        zz        
0       world [ 1         0         0         0         1         0         0
  0         1       ]
1       torso [ 1         0         0         0         1         0         0
  0         1       ]
2 right_thigh [ 1         0         0         0         1         0         0
  0         1       ]
3   right_leg [ 1         0         0         0         1         0         0
  0         1       ]
4  right_foot [ 1         0         0         0         1         0         0
  0         1       ]
5  left_thigh [ 1         0         0         0         1         0         0
  0         1       ]
6    left_leg [ 1         0         0         0         1         0         0
  0         1       ]
7   left_foot [ 1         0         0         0         1         0         0
  0         1       ]

In [16]:
physics.named.data.sensordata

FieldIndexer(sensordata):
0 torso_subtreelinvel [ 0       ]
1 torso_subtreelinvel [ 0       ]
2 torso_subtreelinvel [ 0       ]

In [15]:
physics.named.data.xmat[1:, ['xx', 'xz']]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [18]:
physics.named.data.sensordata['torso_subtreelinvel']

array([0., 0., 0.])

In [19]:
physics.velocity()

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
physics.data.qvel

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
physics.data

In [22]:
physics.data.ptr

In [24]:
physics.named.data.qvel

FieldIndexer(qvel):
0       rootz [ 0       ]
1       rootx [ 0       ]
2       rooty [ 0       ]
3   right_hip [ 0       ]
4  right_knee [ 0       ]
5 right_ankle [ 0       ]
6    left_hip [ 0       ]
7   left_knee [ 0       ]
8  left_ankle [ 0       ]

In [25]:
env.action_spec()

NameError: name 'env' is not defined